In [1]:
%pip install azure-ai-ml azure-identity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.9 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.8/247.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: azure-core
    Found existing installation: azure-core 1.27.1
    Uninstalling azure-core-1.27.1:
      Successfully uninstalled azure-core-1.27.1
  Attempting uninstall: azure-storage-blob
    Found existing installation: azure-storage-blob 12.13.0
    Uninstalling azure-storage-blob-12.13.0:
      Successfully uninstal

In [1]:
# deployment
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

In [2]:
# enter details of your Azure Machine Learning workspace
subscription_id = ""
resource_group = ""
workspace = ""

In [3]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [4]:
import datetime

endpoint_name = "lugtts-endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="This is the endpoint for inferencing text to speech in luganda",
    auth_mode="key"
)

In [40]:
model = Model(path="Conda.yaml")
env = Environment(
    conda_file="Conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="Inference", scoring_script="score.py"
    ),
    instance_type="STANDARD_NC6S_V3",
    instance_count=1,
)

In [41]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Updating local endpoint (lugtts-endpt-09050932965776) .Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32779/score', 'openapi_uri': None, 'name': 'lugtts-endpt-09050932965776', 'description': 'This is the endpoint for inferencing text to speech in luganda', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/oasisaitts/code/Users/2100800280/OasisAiTextToSpeech'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f341c146d30>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [42]:
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

Updating local deployment (lugtts-endpt-09050932965776 / blue) .
Building Docker image from Dockerfile
Step 1/6 : FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
 ---> a79caa0393e2
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Using cache
 ---> 5286c4c3769f
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Using cache
 ---> 0e4d45b04cdc
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> Using cache
 ---> b04334c2c08b
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Using cache
 ---> 4c1daffd08b5
Step 6/6 : CMD ["conda", "run", "--no-capture-output", "-n", "inf-conda-env", "runsvdir", "/var/runit"]
 ---> Using cache
 ---> fb5dfeb89421
Successfully built fb5dfeb89421
Successfully tagged lugtts-endpt-09050932965776:blue

Starting up endpoint.....Done (0m 30s)


ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'lugtts-endpt-09050932965776', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/oasisaitts/code/Users/2100800280/OasisAiTextToSpeech'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f341c1a6730>, 'model': Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': '1d0f1822b9d2d9cdd296f41b2e8b5c4b', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/oasisaitts/code/Users/2100800280/OasisAiTextToSpeech'), 'creation_context': None, 'serialize':

In [43]:
ml_client.online_endpoints.get(name=endpoint_name, local=True)

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32782/score', 'openapi_uri': None, 'name': 'lugtts-endpt-09050932965776', 'description': 'This is the endpoint for inferencing text to speech in luganda', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/oasisaitts/code/Users/2100800280/OasisAiTextToSpeech'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f341c187310>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [44]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, local=True, lines=500
)

"2023-09-05T10:13:18,098964551+00:00 - rsyslog/run \r\n2023-09-05T10:13:18,100753732+00:00 - nginx/run \r\n2023-09-05T10:13:18,105157084+00:00 - gunicorn/run \r\n2023-09-05T10:13:18,109158040+00:00 | gunicorn/run | \r\n2023-09-05T10:13:18,110741023+00:00 | gunicorn/run | ###############################################\r\n2023-09-05T10:13:18,112304406+00:00 | gunicorn/run | AzureML Container Runtime Information\r\n2023-09-05T10:13:18,113717990+00:00 | gunicorn/run | ###############################################\r\n2023-09-05T10:13:18,115139175+00:00 | gunicorn/run | \r\n2023-09-05T10:13:18,537961870+00:00 | gunicorn/run | \r\n2023-09-05T10:13:18,540446843+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20230227.v3\r\n2023-09-05T10:13:18,541689929+00:00 | gunicorn/run | \r\n2023-09-05T10:13:18,542956915+00:00 | gunicorn/run | \r\n2023-09-05T10:13:18,544224302+00:00 | gunicorn/run | PATH environment variable: /opt/miniconda/envs/inf-conda

In [45]:
ml_client.online_endpoints.get(name=endpoint_name, local=True)

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32782/score', 'openapi_uri': None, 'name': 'lugtts-endpt-09050932965776', 'description': 'This is the endpoint for inferencing text to speech in luganda', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/oasisaitts/code/Users/2100800280/OasisAiTextToSpeech'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f341c1bce50>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [46]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="sample-request.json",
    local=True,
)

'{"blob_url": "https://oasisaigenerated.blob.core.windows.net/generatedaudiolug/audio-20230905-101421.wav"}'

In [38]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, local=True, lines=200
)

'  File "/opt/miniconda/envs/inf-conda-env/lib/python3.9/site-packages/azure/core/tracing/decorator.py", line 78, in wrapper_use_tracer\r\n    return func(*args, **kwargs)\r\n  File "/opt/miniconda/envs/inf-conda-env/lib/python3.9/site-packages/azure/storage/blob/_generated/operations/_block_blob_operations.py", line 852, in upload\r\n    map_error(status_code=response.status_code, response=response, error_map=error_map)\r\n  File "/opt/miniconda/envs/inf-conda-env/lib/python3.9/site-packages/azure/core/exceptions.py", line 164, in map_error\r\n    raise error\r\nazure.core.exceptions.ResourceExistsError: The specified blob already exists.\r\nRequestId:ab63d1cb-f01e-0038-45e0-df3f45000000\r\nTime:2023-09-05T10:03:05.3971723Z\r\nErrorCode:BlobAlreadyExists\r\nContent: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobAlreadyExists</Code><Message>The specified blob already exists.\r\nRequestId:ab63d1cb-f01e-0038-45e0-df3f45000000\r\nTime:2023-09-05T10:03:05.3971723Z</Message></Error